### 2.3.7 สร้างโมเดลโดยไลบรารี TF

ในตัวอย่างที่ผ่านมาเราใช้ภาษาไพธอนเขียนโครงสร้างภายในของ DNN ทั้งในส่วนแพร่กระจายด้านหน้าและย้อนหลังเพื่อให้เข้าใจถึงแผนภาพการคำนวณอย่างชัดแจ้ง 
ในทางปฏิบัติเมื่อโมเดลมีความซับซ้อนมากขึ้นจะนิยมใช้ไลบรารีช่วยในการสร้างโมเดล ซึ่ง TF เป็นหนึ่งในไลบรารีที่มีคำสั่งสนับสนุนการสร้างโมเดลตั้งแต่ DNN 
ธรรมดาจนถึงโมเดลการเรียนรู้เชิงลึกประเภทอื่น ในหัวข้อนี้จะสาธิตการสร้างโมเดล DNN สำหรับจำแนกประเภท

**ตัวอย่าง 2.5**

ในตัวอย่างนี้เราจะศึกษาการจำแนกประเภทของภาพสุนัขและแมวจากข้อมูลที่ดาวน์โหลดได้จาก 

https://www.microsoft.com/en-us/download/details.aspx?id=54765 
    
นำไฟล์ไปไว้ใน /datasets และแตกไฟล์ออกเป็นไดเรคทอรีย่อย /kagglecatsanddogs_3367a/Petimages ซึ่งเป็นค่าโดยปริยายของไฟล์ 
kagglecatsanddogs_3367a.zip ที่ดาวน์โหลดมา หากผู้อ่านเปลี่ยนชื่อของไดเรคทอรีก็เพียงแก้โค้ดในตัวอย่างนี้ให้สอดคล้องกัน  
นำเข้าไลบรารีที่ใช้ในการจัดการไฟล์


In [1]:
import os
from shutil import copyfile

ตรวจสอบจำนวนภาพทั้งหมดของแมวและสุนัข

In [ ]:
print(len(os.listdir('datasets/kagglecatsanddogs_3367a/Petimages/Cat/')))
print(len(os.listdir('datasets/kagglecatsanddogs_3367a/Petimages/Dog/')))

สร้างไดเรคทอรีย่อยเพื่อเก็บภาพสำหรับการฝึกและทดสอบดังนี้

In [ ]:
try:
    os.mkdir('datasets/cats-v-dogs')
    os.mkdir('datasets/cats-v-dogs/training')
    os.mkdir('datasets/cats-v-dogs/testing')
    os.mkdir('datasets/cats-v-dogs/training/cats')
    os.mkdir('datasets/cats-v-dogs/training/dogs')
    os.mkdir('datasets/cats-v-dogs/testing/cats')
    os.mkdir('datasets/cats-v-dogs/testing/dogs')
except OSError:
    pass

เขียนฟังก์ชัน split_data() เพื่อแยกจำนวนภาพสำหรับฝึกและทดสอบตามอัตราส่วนที่กำหนดและใส่ไฟล์ลงในไดเรคทอรีย่อยที่สร้างไว้
นอกจากนั้นฟังก์ชันยังสามารถขจัดภาพที่ใช้งานไม่ได้ คือมีขนาดไฟล์เป็นศูนย์ 

In [ ]:
import random
def split_data(SOURCE, TRAINING, TESTING, SPLIT_SIZE):
    files = []
    for filename in os.listdir(SOURCE):
        file = SOURCE + filename
        if os.path.getsize(file) > 0:
            files.append(filename)
        else:
            print(filename + " is zero length, so ignoring.")

    training_length = int(len(files) * SPLIT_SIZE)
    testing_length = int(len(files) - training_length)
    shuffled_set = random.sample(files, len(files))
    training_set = shuffled_set[0:training_length]
    testing_set = shuffled_set[:testing_length]

    for filename in training_set:
        this_file = SOURCE + filename
        destination = TRAINING + filename
        copyfile(this_file, destination)

    for filename in testing_set:
        this_file = SOURCE + filename
        destination = TESTING + filename
        copyfile(this_file, destination)

เรียกฟังก์ชันโดยกำหนด 90% ของจำนวนไฟล์สำหรับการฝึกและ 10% สำหรับทดสอบ เอาต์พุตที่ได้จากการรันฟังก์ชัน split_data() 
คือไฟล์ที่ถูกขจัดออกเนื่องจากมีขนาดเป็นศูนย์ พบว่ามีจำนวนไฟล์เสียสำหรับภาพสุนัขและแมวอย่างละ 1 ไฟล์ 

In [ ]:
CAT_SOURCE_DIR = "datasets/kagglecatsanddogs_3367a/Petimages/Cat/"
TRAINING_CATS_DIR = "datasets/cats-v-dogs/training/cats/"
TESTING_CATS_DIR = "datasets/cats-v-dogs/testing/cats/"
DOG_SOURCE_DIR = "datasets/kagglecatsanddogs_3367a/Petimages/Dog/"
TRAINING_DOGS_DIR = "datasets/cats-v-dogs/training/dogs/"
TESTING_DOGS_DIR = "datasets/cats-v-dogs/testing/dogs/"

split_size = .9
split_data(CAT_SOURCE_DIR, TRAINING_CATS_DIR, TESTING_CATS_DIR, split_size)
split_data(DOG_SOURCE_DIR, TRAINING_DOGS_DIR, TESTING_DOGS_DIR, split_size)

ตรวจสอบจำนวนไฟล์สำหรับฝึกและทดสอบของภาพสุนัขและแมว

In [ ]:
print('Number of training files (Cats) = ' + str(len(os.listdir('datasets/cats-v-dogs/training/cats/'))))
print('Number of training files (Dogs) = ' +str(len(os.listdir('datasets/cats-v-dogs/training/dogs/'))))
print('Number of testing files (Cats) = ' +str(len(os.listdir('datasets/cats-v-dogs/testing/cats/'))))
print('Number of testing files (Dogs) = ' +str(len(os.listdir('datasets/cats-v-dogs/testing/dogs/'))))

หากตรวจสอบข้อมูลภาพเหล่านี้ในไดเรคทอรีจะพบว่ามีขนาดความกว้างความยาวที่แตกต่างกัน  
ดังนั้นก่อนที่จะดำเนินการกับภาพหล่านี้จะต้องมีการประมวลผลเบื้องต้นเพื่อทำให้ข้อมูลมีความเหมาะสมกับการฝึก เช่นปรับมาตราส่วน ขนาดภาพ
ในกรณีที่ไฟล์ภาพถูกเก็บอยู่ในไดเรคทอรีดังเช่นในตัวอย่างนี้ เราสามารถใช้ฟังก์ชันเสริมคือ ImageDataGenerator() เข้าช่วย 
ซึ่งฟังก์ชันสามารถปรับแต่งภาพได้มากกว่านี้เช่นการหมุน ขยาย เฉือน ฯลฯ เรียกว่าการแต่งเติมภาพ (image augmentation)
แต่ในตัวอย่างนี้ต้องการเพียงลดค่าของแต่ละพิกเซลให้อยู่ในช่วง 0 - 1 และปรับขนาดภาพเป็น 150 x 150 

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
TRAINING_DIR = "datasets/cats-v-dogs/training/"
train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(TRAINING_DIR,
                                                    batch_size=100,
                                                    class_mode='binary',
                                                    target_size=(150, 150))
VALIDATION_DIR = "datasets/cats-v-dogs/testing/"
validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_directory(VALIDATION_DIR,
                                                              batch_size=100,
                                                              class_mode='binary',
                                                              target_size=(150, 150))


การสร้างโมเดล DNN โดยไลบรารี TF เริ่มจากนำเข้าไลบรารีที่ต้องการใช้

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import SGD

ทดลองสร้างโมเดล 3 ชั้นแฝง มีจำนวนเซลล์แต่ละชั้นเท่ากับ 128, 64, 1 
ฟังก์ชันกระตุ้นของชั้นแฝงแบบ ReLU และของชั้นเอาต์พุตแบบซิกมอยด์ เขียนเป็นโค้ดได้ดังนี้ 

In [ ]:
model = Sequential([
    Flatten(input_shape=(150, 150, 3)),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(1, activation = 'sigmoid')
])

โดยใช้ Sequential() ในการสร้างโมเดลและต้องใช้ชั้นแรกเป็น Flatten() 
ที่มีรูปแบบอินพุตตรงกับขนาดภาพที่สร้างโดย ImageDataGenerator() 

ใช้เมธอด model.compile() กำหนดตัวหาค่าเหมาะที่สุดแบบ SGD ฟังก์ชันสูญเสีย 'binary_crossentropy'
และใช้ความแม่นยำเป็นตัววัด

In [ ]:
model.compile(optimizer=SGD(learning_rate=0.01),loss='binary_crossentropy', metrics=['accuracy'])

ฝึกโมเดลจำนวน 15 รอบ โดยคืนค่าข้อมูลที่เป็นประวัติการฝึกในตัวแปรดิกชันนารี history (เซลล์นี้อาจใช้เวลานานในการรัน)

In [ ]:
history = model.fit(train_generator, epochs=15,verbose=1,validation_data=validation_generator)

xห่ลังจากการฝึกเสร็จสิ้น เราสามารถเข้าถึงข้อมูลใน history เพื่อพล็อตค่าความแม่นยำและการสูญเสียเทียบกับจำนวนรอบการฝึก 
ดังแสดงในรููปที่ 2.23 และ 2.24 ตามลำดับ

In [ ]:
acc=history.history['accuracy']
val_acc=history.history['val_accuracy']
loss=history.history['loss']
val_loss=history.history['val_loss']

epochs=range(len(acc)) # Get number of epochs

#------------------------------------------------
# Plot training and validation accuracy per epoch
#------------------------------------------------
plt.figure()
plt.plot(epochs, acc, 'r')
plt.plot(epochs, val_acc, 'b')
plt.title('Training and validation accuracy')
plt.ylim([0.5,0.7])
plt.legend(["Training","Validation"])
plt.show()


รูปที่ 2.23 ค่าความแม่นยำจากข้อมูลการฝึกและข้อมูลทดสอบเทียบกับจำนวนรอบ

In [ ]:
#------------------------------------------------
# Plot training and validation loss per epoch
#------------------------------------------------
plt.figure()
plt.plot(epochs, loss, 'r')
plt.plot(epochs, val_loss, 'b')
plt.ylim([0.6,0.7])
plt.legend(["Training","Validation"])
plt.show()


รูปที่ 2.24 ค่าจากฟังก์ชันสูญเสียเทียบกับจำนวนรอบ

จากความแม่นยำที่ได้พบว่ามีค่าอยู่ประมาณไม่เกิน 70% ซึ่งเป็นผลที่ไม่น่าพอใจนักสำหรับการจำแนกภาพแมวและสุนัข 
ข้อด้อยของการใช้ DNN ในการจำแนกภาพคือไม่สามารถเรียนรู้ข้อมูลเชิงพื้นที่ (spatial information) 
ซึ่งเป็นองค์ประกอบภาพที่มีความสำคัญ เมื่อเราได้ศึกษาโมเดลแบบ CNN จะพบว่าสามารถจำแนกภาพด้วยความแม่นยำกว่า
DNN มาก


<p align="center">
<img src="https://drive.google.com/thumbnail?id=13bzT7Rmy3bzvE7TiS0yfQo94kpxMuipF" alt="dewninja"/>
</p>
<div align="center">dew.ninja 2022</div>